<a href="https://colab.research.google.com/github/withpi/cookbook-withpi/blob/main/workshop/Building_Evals_that_actually_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://withpi.ai"><img src="https://withpi.ai/logo/logoFullBlack.svg" width="240"></a>

<a href="https://code.withpi.ai"><font size="4">Documentation</font></a>

<a href="https://withpi.ai"><font size="4">Copilot</font></a>

# [Workshop] - Building Evals that actually work

This colab walks you through scoring data programmatically and using it in a few different workflows.

It uses similar data to the Google Sheets workflow [Building Evals that actually work](https://docs.google.com/spreadsheets/d/1AfBIWpIr0wUIpHnPEMXV7Rh_qGEsx-dxyzHhavX_xgU/edit?gid=2058777572#gid=2058777572), but it uses Python code snippets to go one level deeper.

## Install and initialize SDK

You'll need a `WITHPI_API_KEY` from https://withpi.ai/account/keys.  Add it to your notebook secrets (the key symbol) on the left.

Run the cell below to install packages and load the SDK

In [1]:
%%capture

%pip install withpi withpi-utils litellm
# See https://github.com/huggingface/datasets/issues/7570
%pip install -U datasets huggingface_hub fsspec

import os
from google.colab import userdata
from withpi import PiClient

# Load the notebook secret into the environment so the Pi Client can access it.
os.environ["WITHPI_API_KEY"] = userdata.get('WITHPI_API_KEY')

pi = PiClient()

## Setup scoring system

From your Copilot session with your scoring system, click `Code`, and then copy the contents of that cell.  Paste it below, overwriting the example spec.  This will define a `score()` function that the rest of the Colab will refer to.

In [2]:
from withpi import PiClient

# Initialize Pi client
pi = PiClient()

def score(llm_input, llm_output)->float:
  scoring_spec =   [
    {
      "label": "Title Accuracy",
      "question": "Does the output include an accurate and relevant title summarizing the transcript?",
      "weight": 0.5
    },
    {
      "label": "Key Insights",
      "question": "Does the output extract all major insights from the transcript?",
      "weight": 0.5
    },
    {
      "label": "Action Items",
      "question": "Are all action items mentioned in the transcript included in the output?",
      "weight": 0.5
    },
    {
      "label": "Action Details",
      "question": "Does the output provide optional details like owner and due date for action items when available?",
      "weight": 0.5
    },
    {
      "label": "JSON Format",
      "question": "Is the output structured correctly in JSON format?",
      "python_code": "from typing import Any\n\ndef score(\n    response_text: str,\n    input_text: str,\n    kwargs: dict[str, Any],\n) -> dict:\n\n    def evaluate_response(response):\n        \"\"\"\n        Check if the response is structured correctly in JSON format.\n        \n        Args:\n            response (str): The LLM's response text\n        \n        Returns:\n            float: 1.0 if the response is valid JSON, 0.0 otherwise\n        \"\"\"\n        import json\n    \n        try:\n            # Attempt to parse the response as JSON\n            json.loads(response)\n            return 1.0  # Valid JSON\n        except json.JSONDecodeError:\n            return 0.0  # Invalid JSON\n        except Exception:\n            return 0.0  # Handle any other unexpected errors gracefully\n\n    final_score = evaluate_response(response_text)\n    return {\"score\": final_score, \"explanation\": \"\"}\n",
      "scoring_type": "PYTHON_CODE",
      "weight": 0.5
    },
    {
      "label": "Transcript Coverage",
      "question": "Does the output cover all relevant sections of the transcript without omitting critical information?",
      "weight": 0.3
    },
    {
      "label": "Insight Clarity",
      "question": "Are the key insights presented in a clear and concise manner?",
      "weight": 0.3
    },
    {
      "label": "Action Completeness",
      "question": "Does each action item include sufficient details to be actionable?",
      "weight": 0.3
    },
    {
      "label": "Owner Identification",
      "question": "Does the output correctly identify the owner for each action item when specified in the transcript?",
      "weight": 0.3
    },
    {
      "label": "Due Date Inclusion",
      "question": "Does the output include due dates for action items when specified in the transcript?",
      "weight": 0.3
    },
    {
      "label": "Redundancy Avoidance",
      "question": "Does the output avoid redundant or repetitive information?",
      "weight": 0.1
    },
    {
      "label": "Abusive Language",
      "question": "Does the output avoid using abusive or inappropriate language?",
      "weight": 0.1
    }
  ]

  return pi.scoring_system.score(llm_input=llm_input, llm_output=llm_output, scoring_spec=scoring_spec).total_score


## Load a dataset

Load some data from Hugging Face to play around with.  This is the same data as was in the Sheet from before

In [5]:
from datasets import load_dataset

ds = load_dataset("withpi/aiewf_workshop_data")

display(ds)

README.md:   0%|          | 0.00/541 [00:00<?, ?B/s]

(…)-00000-of-00001-e703f8357ce85213.parquet:   0%|          | 0.00/185k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Rating', 'Raw meeting transcript', 'Extracted output'],
        num_rows: 126
    })
})

In [6]:
import pandas as pd

def score_example(example):
  example["Score"] = score(example["Raw meeting transcript"], example["Extracted output"])
  return example

scored = ds["train"].map(score_example)
display(pd.DataFrame(scored))

Parameter 'function'=<function score_example at 0x7f103835a480> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/126 [00:00<?, ? examples/s]

,Rating,Raw meeting transcript,Extracted output,Score
0,thumbs_down,Speaker 0: Next item is the one that you're an...,"```json\n{\n ""title"": ""Discussion on Belmont ...",0.0000
1,thumbs_down,Speaker 0: Next item is the one that you're an...,"```json\n{\n ""title"": ""Adoption of Annual Spe...",0.0000
2,thumbs_up,Speaker 2: Thank you. Item 15.\nSpeaker 0: Rep...,"{\n ""title"": ""Adoption of Ordinance Amending ...",0.8838
3,thumbs_down,Speaker 2: Thank you. Item 15.\nSpeaker 0: Rep...,"```json\n{\n ""title"": ""Marijuana Dispensary H...",0.0000
4,thumbs_down,"Speaker 1: Madam Clerk, could you please read ...","```json\n{\n ""title"": ""Boston Commemoration C...",0.0000
...,...,...,...,...
121,thumbs_up,Speaker 0: Item 53 recommendation to increase ...,"```json\n{\n ""title"": ""City Council Appropria...",0.0000
122,thumbs_up,Speaker 0: Item 53 recommendation to increase ...,"```json\n{\n ""title"": ""City Council Appropria...",0.0000
123,thumbs_up,"Speaker 4: Thank you, Madam Clerk. The motion ...","{\n ""title"": ""Approval of Council Bill for Ge...",0.0178
124,thumbs_up,"Speaker 4: Thank you, Madam Clerk. The motion ...","```json\n{\n ""title"": ""Approval of Georgetown...",0.0000


## Rejection Sampling

Now that you have some measure of goodness, it's easy to throw out bad results, to, for example, build a training set.

In [ ]:
filtered = scored.filter(lambda example: example["Score"] > 0.7)
display(pd.DataFrame(filtered))

Filter:   0%|          | 0/151 [00:00<?, ? examples/s]

,Length,Manual Rating,Prompt,Generated Tweet,Score
0,228,"Excellent: Clear value proposition, specific b...",Write me a product marketing tweet for an AI a...,"Plan your dream getaway in minutes, not hours....",0.826823
1,227,"Excellent: Clear value proposition, specific b...",Write me a product marketing tweet for an AI a...,From wishlist to itinerary in 60 seconds. Our ...,0.790365
2,184,"Excellent: Clear value proposition, specific b...",Write me a product marketing tweet for an AI a...,Introducing a smarter way to travel! Our app c...,0.742188
3,252,"Excellent: Clear value proposition, specific b...",Write me a product marketing tweet for an AI a...,Travelers are saving 5+ hours of planning time...,0.815104
4,236,"Excellent: Clear value proposition, specific b...",Write me a product marketing tweet for an AI a...,Travel planning shouldn't be a full-time job. ...,0.854167
5,167,"Average: Basic information but generic, lacks ...",Write me a product marketing tweet for an AI a...,Planning a trip? Our software makes it easy! C...,0.765625
6,178,None,Write me a product marketing tweet for an AI a...,The trip planning revolution has arrived! Our ...,0.721354
7,212,None,Write me a product marketing tweet for an AI a...,Ever spent hours planning a trip only to wonde...,0.779948
8,228,None,Write me a product marketing tweet for an AI a...,I planned my entire European vacation in under...,0.752604
9,211,None,Write me a product marketing tweet for an AI a...,"The only trip planner with a ""Serendipity Scor...",0.808594


## Model Comparison

Now that you have a scoring system, let's try a few different models to more objectively compare how they do.

We have a dataset of longer meeting transcripts that are somewhat more confusing than average.  Let's see how a couple of different models do.

You can import a Google Gemini key from AI Studio on the left pane, which populates a `GOOGLE_API_KEY` secret.  At low rates it's free.  Or adjust to a model of your choice with a key using docs at https://docs.litellm.ai/docs/.

In [7]:
from datasets import load_dataset

ds = load_dataset("withpi/aiewf_workshop_data_extra_long_transcripts")

display(ds)

DatasetDict({
    train: Dataset({
        features: ['Raw meeting transcript'],
        num_rows: 49
    })
})

In [10]:
import litellm

os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

good_prompt = """
You are an AI assistant designed to analyze meeting or conversation transcripts. Given a transcript, your task is to extract the following elements and return them in JSON format:

Title: A concise and informative title summarizing the main topic or purpose of the conversation.
Key Insights: A list of the most important insights, takeaways, or conclusions drawn from the transcript. Each insight should be clear, standalone, and phrased in natural language.
Action Items: A list of actionable tasks or follow-ups mentioned or implied in the conversation. Include who is responsible (if mentioned), what the action is, and any relevant deadlines or context.

Guidelines:
Do not include any information not found in or inferred directly from the transcript. Be concise and clear in phrasing each output field.
Output only valid JSON. Do not include any additional text or markdown formatting. Output only a valid JSON object.

JSON Output Format:
{
  "title": "string",
  "key_insights": [ "string", "string", ... ],
  "action_items": [
    { "description": "string", "owner": "string (optional)", "due_date": "string (optional)" }
  ]
}
"""

def generate_and_score(model, example):
  example[f"{model} summary"] = litellm.completion(
    model=model,
    messages=[
        {"content": good_prompt, "role": "system"},
        {"content": example["Raw meeting transcript"], "role": "user"}
    ]).choices[0].message.content
  example[f"{model} Score"] = score(example["Raw meeting transcript"], example[f"{model} summary"])
  return example

#sample = ds['train'].select(range(5)).map(lambda ex: generate_and_score("gemini/gemini-2.5-flash-preview-05-20", ex))
sample = sample.select(range(5)).map(lambda ex: generate_and_score("gemini/gemini-1.5-flash", ex))

display(pd.DataFrame(sample))

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

,Raw meeting transcript,gemini/gemini-2.5-flash-preview-05-20 tweet,gemini/gemini-2.5-flash-preview-05-20 Score,gemini/gemini-1.5-flash-8b tweet,gemini/gemini-1.5-flash-8b Score,gemini/gemini-1.5-flash tweet,gemini/gemini-1.5-flash Score
0,Speaker 0: Ten Eyes Cancer build 20 1-659 has ...,"```json\n{\n ""title"": ""Council Bill 21-720 Pa...",0.0,"```json\n{\n ""title"": ""Public Hearing and App...",0.0,"```json\n{\n ""title"": ""Approval of Map Amendm...",0.0
1,"Speaker 1: Thank you. Thank you, Miss Butler. ...","```json\n{\n ""title"": ""Homelessness Services ...",0.0,"```json\n{\n ""title"": ""Alameda City Council M...",0.0,"```json\n{\n ""title"": ""Allocation of American...",0.0
2,Speaker 0: Motion case.\nSpeaker 1: Thank you....,"```json\n{\n ""title"": ""Denial of Easy Greens ...",0.0,"```json\n{\n ""title"": ""Hearing on Easy Greens...",0.0,"```json\n{\n ""title"": ""Appeal Hearing: Denial...",0.0
3,Speaker 1: Thank you very much. Moving on to t...,"```json\n{\n ""title"": ""Long Beach Green Fleet...",0.0,"```json\n{\n ""title"": ""Green Fleet Program Up...",0.0,"```json\n{\n ""title"": ""Long Beach Green Fleet...",0.0
4,"Speaker 2: Motion carries, we'll.\nSpeaker 1: ...","```json\n{\n ""title"": ""City Council Report on...",0.0,"```json\n{\n ""title"": ""Placemaking Initiative...",0.0,"```json\n{\n ""title"": ""Place Make the Vote In...",0.0


## Test system prompts

The worked example so far embeds the task into the prompt itself and uses a simple instruction following system prompt.

You can also try varying the system prompt to see how that influences scoring.

In [ ]:
import litellm
from datasets import Dataset

from IPython.display import display, Markdown, HTML

os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')

bad_prompt = """
Your job is to summarize meetings. Include title, key insights, and action items. Output as json. Do not surround the JSON in triple ticks or any other decoration.
"""

good_prompt = """
You are an AI assistant designed to analyze meeting or conversation transcripts. Given a transcript, your task is to extract the following elements and return them in JSON format:

Title: A concise and informative title summarizing the main topic or purpose of the conversation.
Key Insights: A list of the most important insights, takeaways, or conclusions drawn from the transcript. Each insight should be clear, standalone, and phrased in natural language.
Action Items: A list of actionable tasks or follow-ups mentioned or implied in the conversation. Include who is responsible (if mentioned), what the action is, and any relevant deadlines or context.

Guidelines:
Do not include any information not found in or inferred directly from the transcript. Be concise and clear in phrasing each output field.
Output only valid JSON. Do not include any additional text or markdown formatting. Output only a valid JSON object.

JSON Output Format:
{
  "title": "string",
  "key_insights": [ "string", "string", ... ],
  "action_items": [
    { "description": "string", "owner": "string (optional)", "due_date": "string (optional)" }
  ]
}
"""

inputs = pd.DataFrame(load_dataset('withpi/aiewf_workshop_data')['train'].filter(lambda ex: ex["Rating"] == "thumbs_down"))
inputs = inputs['Raw meeting transcript'].iloc[:5]

def generate_and_score(system_prompt, input):
  response = litellm.completion(
    model="gemini/gemini-2.5-flash-preview-05-20",
    messages=[
      {"content": system_prompt, "role": "system"},
      {"content": input, "role": "user"}
    ]).choices[0].message.content
  score_ = score(input, response)
  print(f'Response ({score_}):', response)
  return { 'response': response, 'score': score_ }

results = { 'Bad Prompt': [], 'Good Prompt': [] }
for input in inputs:
  bad = generate_and_score(bad_prompt, input)
  good = generate_and_score(good_prompt, input)
  results['Bad Prompt'].append(bad['score'])
  results['Good Prompt'].append(good['score'])
results = pd.DataFrame({ 'Inputs': inputs, **results })
display(results)
display(results[['Bad Prompt', 'Good Prompt']].mean())

Question scores {'Title Accuracy': 0.9961, 'Key Insights': 0.957, 'Action Items': 0.9688, 'Action Details': 0.2256, 'Transcript Coverage': 0.9531, 'Insight Clarity': 0.8711, 'Action Completeness': 0.9648, 'Owner Identification': 0.9961, 'Due Date Inclusion': 0.0664, 'Redundancy Avoidance': 0.9688, 'Abusive Language': 1.0}
Response (0.3795): ```json
{
  "title": "Community Facility District (CFD) Annual Special Tax Increase Approval",
  "key_insights": [
    "A resolution was proposed to adopt an annual special tax for Community Facility District (CFD) Number 2007-24, applicable to commercial properties in Belmont Shore for fiscal year 2019.",
    "The special tax rate, which has been $0.12 per commercial square foot since 2006, is proposed to be increased. This is the first increase in approximately 12 years.",
    "The primary reasons for the proposed increase are the rising costs associated with administering the various payments, servicing the bond, and increased annual debt payment

,Inputs,Bad Prompt,Good Prompt
0,Speaker 0: Next item is the one that you're an...,0.3795,0.1336
1,Speaker 0: Next item is the one that you're an...,0.1136,0.9783
2,Speaker 2: Thank you. Item 15.\nSpeaker 0: Rep...,0.0281,0.8413
3,"Speaker 1: Madam Clerk, could you please read ...",0.0359,0.0470
4,"Speaker 1: Madam Clerk, could you please read ...",0.0415,0.0450


,0
Bad Prompt,0.11972
Good Prompt,0.40904
